In [13]:
from drn_interactions.config import Config, ExperimentInfo
from drn_interactions.io import load_derived_generic
from drn_interactions.transforms.brain_state import StateHandler, RawEEGHandler
from drn_interactions.responders.brain_state import SpikeRateResonders
from drn_interactions.transforms.spikes import SpikesHandler
from drn_interactions.transforms.brain_state_spikes import align_bins_to_states_long
from drn_interactions.stats import se_mean, se_prop

import pandas as pd
import numpy as np
from IPython.display import display
from pprint import pprint

In [57]:
def mean_se(x):
    return f"N={len(x)}; µ={x.mean():.2f} +- {se_mean(x):.2f}"

In [58]:
eeg_sessions = ExperimentInfo.eeg_sessions
neurons = load_derived_generic("neuron_types.csv").query("session_name in @eeg_sessions")

by_session = neurons.groupby(["session_name", "neuron_type"]).apply(len)
mean_by_nt = by_session.reset_index().groupby("neuron_type")[0].apply(mean_se)

out = dict(
    num_eeg_sessions = len(eeg_sessions),
    num_neurons = mean_se(by_session),
    num_neurons_by_nt = mean_by_nt.to_dict(),
)

pprint(out)

{'num_eeg_sessions': 17,
 'num_neurons': 'N=43; µ=7.40 +- 0.79',
 'num_neurons_by_nt': {'FF': 'N=13; µ=5.23 +- 0.91',
                       'SIR': 'N=15; µ=7.33 +- 1.25',
                       'SR': 'N=15; µ=9.33 +- 1.58'}}


In [59]:
# proportion of time in each state
import mchmm as mc
from drn_interactions.transforms.brain_state import StateHandler


states_handler = StateHandler(
    quality_to_include=["good", "med"]
    )
states = states_handler.states_df

In [60]:
def markov_chain(ser: pd.Series) -> pd.DataFrame:
    mod = mc.MarkovChain().from_data(ser.values)
    df_res = (
        pd.DataFrame(mod.observed_p_matrix, index=mod.states, columns=mod.states)
        .reset_index()
        .rename(columns={"index": "from"})
        .melt(id_vars="from", var_name="to", value_name="p")
    )
    return df_res

def markov_chain_by_session(df, state_col="state", session_col="session_name"):
    df_res = df.groupby(session_col)[state_col].apply(markov_chain)
    df_res = df_res.reset_index().drop("level_1", axis=1)
    return df_res


df_res = markov_chain_by_session(states)
df_res.pivot_table(index="from", columns="to", values="p", aggfunc=mean_se)

to,act,sw
from,,
act,N=14; µ=0.97 +- 0.01,N=14; µ=0.03 +- 0.01
sw,N=14; µ=0.03 +- 0.01,N=14; µ=0.97 +- 0.01


In [61]:
# Change in spike rate for each neuron type betwee states

df_responders_rate = (
    load_derived_generic("brain_states_spikerate_responders.csv")
    .merge(neurons)
)
avg_spikerate_change = mean_se(df_responders_rate.loc[lambda x: x.sig]["Diff_raw"])
avg_spikerate_change_by_unit = df_responders_rate.loc[lambda x: x.sig].groupby(["neuron_type"])["Diff_raw"].apply(mean_se)

avg_spikerate_change_z = mean_se(df_responders_rate.loc[lambda x: x.sig]["Diff"])
avg_spikerate_change_by_unit_z = df_responders_rate.loc[lambda x: x.sig].groupby(["neuron_type"])["Diff"].apply(mean_se)


out = dict(
    avg_spikerate_change=avg_spikerate_change,
    avg_spikerate_change_by_unit=avg_spikerate_change_by_unit.to_dict(),
    avg_spikerate_change_z=avg_spikerate_change_z,
    avg_spikerate_change_by_unit_z=avg_spikerate_change_by_unit_z.to_dict(),
)
pprint(out)

{'avg_spikerate_change': 'N=244; µ=1.29 +- 0.38',
 'avg_spikerate_change_by_unit': {'FF': 'N=55; µ=1.64 +- 0.89',
                                  'SIR': 'N=81; µ=1.44 +- 0.68',
                                  'SR': 'N=108; µ=0.99 +- 0.51'},
 'avg_spikerate_change_by_unit_z': {'FF': 'N=55; µ=0.18 +- 0.12',
                                    'SIR': 'N=81; µ=0.21 +- 0.08',
                                    'SR': 'N=108; µ=0.26 +- 0.08'},
 'avg_spikerate_change_z': 'N=244; µ=0.22 +- 0.05'}


In [62]:
# Proportion of each unit type phase locked by oscillation
from drn_interactions.spiketrains.neurontype_props import ChiSquarePostHoc

df_prefered_angles = load_derived_generic("brain_states_phase_responders.csv")

df_prefered_angles = df_prefered_angles.assign(sig=lambda x: x.p < 0.05)
mod = ChiSquarePostHoc(value_col="sig", round=2)

display(mod(df_prefered_angles))
display(mod(df_prefered_angles.query("oscillation == 'delta'")))
display(mod(df_prefered_angles.query("oscillation == 'theta'")))

anova                         Chi2(2)=32.0 (p=0.00*)
SIR - FF     41.4%; 72.39% | Chi(1.0)=30.62 (p=0.0*)
SIR - SR      41.4%; 54.78% | Chi(1.0)=8.08 (p=0.0*)
FF - SR     72.39%; 54.78% | Chi(1.0)=10.93 (p=0.0*)
dtype: object

anova                         Chi2(2)=16.4 (p=0.00*)
SIR - FF    57.94%; 86.76% | Chi(1.0)=14.86 (p=0.0*)
SIR - SR     57.94%; 64.23% | Chi(1.0)=0.76 (p=0.38)
FF - SR     86.76%; 64.23% | Chi(1.0)=10.29 (p=0.0*)
dtype: object

anova                        Chi2(2)=19.9 (p=0.00*)
SIR - FF    25.0%; 57.58% | Chi(1.0)=17.21 (p=0.0*)
SIR - SR     25.0%; 45.19% | Chi(1.0)=9.73 (p=0.0*)
FF - SR     57.58%; 45.19% | Chi(1.0)=2.25 (p=0.13)
dtype: object